# Async Programming in Python for Backend Development

**A comprehensive guide to asynchronous programming patterns, best practices, and real-world applications**

---

## Table of Contents

1. [Why Async Matters for Backend Development](#1-why-async-matters)
2. [Python asyncio Fundamentals](#2-asyncio-fundamentals)
3. [async/await Syntax and Patterns](#3-async-await-patterns)
4. [Running Concurrent Tasks](#4-concurrent-tasks)
5. [Async Context Managers and Iterators](#5-async-context-managers)
6. [Error Handling in Async Code](#6-error-handling)
7. [Common Pitfalls](#7-common-pitfalls)
8. [Async Libraries Ecosystem](#8-async-ecosystem)
9. [FastAPI Async Patterns](#9-fastapi-patterns)
10. [Testing Async Code](#10-testing-async)
11. [When NOT to Use Async](#11-when-not-to-use)

---

<a id="1-why-async-matters"></a>
## 1. Why Async Matters for Backend Development

### Understanding I/O Bound vs CPU Bound Operations

Backend applications spend most of their time waiting for external resources:

| Operation Type | Examples | Characteristics | Solution |
|---------------|----------|-----------------|----------|
| **I/O Bound** | Database queries, HTTP requests, File I/O, Network calls | Waiting for external systems | Async programming |
| **CPU Bound** | Image processing, Encryption, ML inference, Data parsing | Heavy computation | Multiprocessing |

### The Problem with Synchronous Code

```
Synchronous Execution (1 worker, 3 requests):
┌─────────────────────────────────────────────────────────────────────────────┐
│ Request 1: [████████████████] (2s)                                          │
│ Request 2:                   [████████████████] (2s)                        │
│ Request 3:                                      [████████████████] (2s)     │
└─────────────────────────────────────────────────────────────────────────────┘
Total time: 6 seconds

Asynchronous Execution (1 worker, 3 requests):
┌─────────────────────────────────────────────────────────────────────────────┐
│ Request 1: [████████████████] (2s)                                          │
│ Request 2: [████████████████] (2s)                                          │
│ Request 3: [████████████████] (2s)                                          │
└─────────────────────────────────────────────────────────────────────────────┘
Total time: ~2 seconds (concurrent execution)
```

In [ ]:
import time
import asyncio
from typing import List

# Synchronous approach - sequential execution
def fetch_data_sync(item_id: int) -> dict:
    """Simulates a database/API call that takes 1 second."""
    time.sleep(1)  # Blocking call
    return {"id": item_id, "data": f"Result for {item_id}"}

def process_items_sync(item_ids: List[int]) -> List[dict]:
    """Process items sequentially."""
    results = []
    for item_id in item_ids:
        results.append(fetch_data_sync(item_id))
    return results

# Measure synchronous execution
start = time.perf_counter()
sync_results = process_items_sync([1, 2, 3, 4, 5])
sync_duration = time.perf_counter() - start
print(f"Synchronous: {sync_duration:.2f}s for {len(sync_results)} items")

In [ ]:
# Asynchronous approach - concurrent execution
async def fetch_data_async(item_id: int) -> dict:
    """Simulates an async database/API call that takes 1 second."""
    await asyncio.sleep(1)  # Non-blocking call
    return {"id": item_id, "data": f"Result for {item_id}"}

async def process_items_async(item_ids: List[int]) -> List[dict]:
    """Process items concurrently."""
    tasks = [fetch_data_async(item_id) for item_id in item_ids]
    results = await asyncio.gather(*tasks)
    return list(results)

# Measure asynchronous execution
start = time.perf_counter()
async_results = await process_items_async([1, 2, 3, 4, 5])
async_duration = time.perf_counter() - start
print(f"Asynchronous: {async_duration:.2f}s for {len(async_results)} items")
print(f"\nSpeedup: {sync_duration / async_duration:.1f}x faster")

### Real-World Backend Scenario: API Gateway

Consider an API endpoint that aggregates data from multiple services:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           API Gateway Request                               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌──────────────┐    ┌──────────────┐    ┌──────────────┐                 │
│   │ User Service │    │ Order Service│    │ Product Svc  │                 │
│   │   (50ms)     │    │   (100ms)    │    │   (75ms)     │                 │
│   └──────────────┘    └──────────────┘    └──────────────┘                 │
│                                                                             │
│   Sync Total:  50 + 100 + 75 = 225ms                                       │
│   Async Total: max(50, 100, 75) = 100ms                                    │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

<a id="2-asyncio-fundamentals"></a>
## 2. Python asyncio Fundamentals

### Core Concepts Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                            ASYNCIO ARCHITECTURE                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                         EVENT LOOP                                   │  │
│   │  (Single-threaded scheduler that manages all async operations)      │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                    │                                                        │
│         ┌──────────┴──────────┬─────────────────────┐                      │
│         ▼                     ▼                     ▼                      │
│   ┌───────────┐         ┌───────────┐         ┌───────────┐               │
│   │ COROUTINE │         │   TASK    │         │  FUTURE   │               │
│   │           │         │           │         │           │               │
│   │ async def │         │ Wrapped   │         │ Low-level │               │
│   │ function  │ ──────▶ │ coroutine │ ──────▶ │ awaitable │               │
│   │           │  create │ scheduled │  result │ result    │               │
│   └───────────┘  _task  └───────────┘  holder └───────────┘               │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.1 The Event Loop

The event loop is the heart of asyncio - it schedules and runs async code.

In [ ]:
import asyncio

# Getting the current event loop
loop = asyncio.get_event_loop()
print(f"Event Loop: {type(loop).__name__}")
print(f"Is running: {loop.is_running()}")

# In Jupyter/IPython, the event loop is already running
# In scripts, you'd use asyncio.run() which creates and manages the loop

async def demonstrate_loop():
    """Show event loop details from within a coroutine."""
    current_loop = asyncio.get_running_loop()
    print(f"Running loop: {current_loop}")
    print(f"Loop time: {current_loop.time():.4f}")
    return "Loop demonstration complete"

result = await demonstrate_loop()
print(result)

### 2.2 Coroutines

Coroutines are the building blocks of async code - special functions that can be paused and resumed.

In [ ]:
import asyncio
import inspect

# Define a coroutine function
async def my_coroutine(name: str, delay: float) -> str:
    """A simple coroutine that simulates async work."""
    print(f"[{name}] Starting...")
    await asyncio.sleep(delay)  # Yield control back to event loop
    print(f"[{name}] Completed after {delay}s")
    return f"{name} result"

# Calling the function returns a coroutine object (not the result!)
coro = my_coroutine("test", 0.1)
print(f"Type: {type(coro)}")
print(f"Is coroutine: {inspect.iscoroutine(coro)}")
print(f"Is coroutine function: {inspect.iscoroutinefunction(my_coroutine)}")

# To get the result, we must await it
result = await coro
print(f"Result: {result}")

### 2.3 Tasks

Tasks wrap coroutines and schedule them for execution. They allow coroutines to run concurrently.

In [ ]:
import asyncio

async def worker(name: str, duration: float) -> str:
    """Simulates an async worker."""
    print(f"Worker {name}: started")
    await asyncio.sleep(duration)
    print(f"Worker {name}: finished")
    return f"Result from {name}"

async def demonstrate_tasks():
    """Show how tasks enable concurrent execution."""
    
    # Create tasks - they start running immediately!
    task1 = asyncio.create_task(worker("A", 2), name="TaskA")
    task2 = asyncio.create_task(worker("B", 1), name="TaskB")
    task3 = asyncio.create_task(worker("C", 1.5), name="TaskC")
    
    print(f"\nTask 1: {task1.get_name()}, done={task1.done()}")
    print(f"Task 2: {task2.get_name()}, done={task2.done()}")
    print(f"Task 3: {task3.get_name()}, done={task3.done()}")
    
    # Wait for all tasks to complete
    results = await asyncio.gather(task1, task2, task3)
    
    print(f"\nAll tasks completed!")
    print(f"Results: {results}")
    
    return results

await demonstrate_tasks()

### 2.4 Futures

Futures are low-level awaitables representing an eventual result. Tasks are actually a subclass of Future.

In [ ]:
import asyncio

async def demonstrate_futures():
    """Show how Futures work at a low level."""
    loop = asyncio.get_running_loop()
    
    # Create a Future manually
    future = loop.create_future()
    
    print(f"Future created: done={future.done()}")
    
    # Simulate setting the result from somewhere else
    async def set_result_later():
        await asyncio.sleep(1)
        future.set_result("Future completed!")
        print("Result was set on the future")
    
    # Start the task that will set the result
    asyncio.create_task(set_result_later())
    
    # Wait for the future
    print("Waiting for future...")
    result = await future
    print(f"Got result: {result}")
    
    # Demonstrate Task is a Future subclass
    task = asyncio.create_task(asyncio.sleep(0.1))
    print(f"\nTask is Future subclass: {isinstance(task, asyncio.Future)}")

await demonstrate_futures()

---

<a id="3-async-await-patterns"></a>
## 3. async/await Syntax and Patterns

### 3.1 Basic Patterns

In [ ]:
import asyncio
from typing import Any, Optional

# Pattern 1: Simple async function
async def fetch_user(user_id: int) -> dict:
    """Fetch user from database."""
    await asyncio.sleep(0.1)  # Simulate DB query
    return {"id": user_id, "name": f"User_{user_id}"}

# Pattern 2: Async function with error handling
async def safe_fetch_user(user_id: int) -> Optional[dict]:
    """Fetch user with error handling."""
    try:
        if user_id < 0:
            raise ValueError("Invalid user ID")
        return await fetch_user(user_id)
    except Exception as e:
        print(f"Error fetching user {user_id}: {e}")
        return None

# Pattern 3: Chaining async calls
async def fetch_user_orders(user_id: int) -> dict:
    """Fetch user and their orders."""
    user = await fetch_user(user_id)
    
    # Simulate fetching orders
    await asyncio.sleep(0.1)
    orders = [{"order_id": i, "user_id": user_id} for i in range(3)]
    
    return {"user": user, "orders": orders}

# Test the patterns
user = await fetch_user(1)
print(f"Pattern 1 - User: {user}")

safe_user = await safe_fetch_user(-1)
print(f"Pattern 2 - Safe fetch: {safe_user}")

user_orders = await fetch_user_orders(1)
print(f"Pattern 3 - User with orders: {user_orders}")

### 3.2 Advanced Patterns

In [ ]:
import asyncio
from typing import Callable, TypeVar, Awaitable
from functools import wraps
import time

T = TypeVar('T')

# Pattern 4: Async decorator for timing
def async_timed(func: Callable[..., Awaitable[T]]) -> Callable[..., Awaitable[T]]:
    """Decorator to time async functions."""
    @wraps(func)
    async def wrapper(*args, **kwargs) -> T:
        start = time.perf_counter()
        result = await func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(f"{func.__name__} took {elapsed:.4f}s")
        return result
    return wrapper

@async_timed
async def slow_operation() -> str:
    """A slow async operation."""
    await asyncio.sleep(0.5)
    return "Done!"

# Pattern 5: Retry decorator with exponential backoff
def async_retry(
    max_attempts: int = 3,
    base_delay: float = 1.0,
    exponential_base: float = 2.0
):
    """Decorator for retrying failed async operations."""
    def decorator(func: Callable[..., Awaitable[T]]) -> Callable[..., Awaitable[T]]:
        @wraps(func)
        async def wrapper(*args, **kwargs) -> T:
            last_exception = None
            for attempt in range(max_attempts):
                try:
                    return await func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    if attempt < max_attempts - 1:
                        delay = base_delay * (exponential_base ** attempt)
                        print(f"Attempt {attempt + 1} failed, retrying in {delay}s...")
                        await asyncio.sleep(delay)
            raise last_exception
        return wrapper
    return decorator

attempt_count = 0

@async_retry(max_attempts=3, base_delay=0.1)
async def flaky_operation() -> str:
    """Simulates an operation that fails intermittently."""
    global attempt_count
    attempt_count += 1
    if attempt_count < 3:
        raise ConnectionError("Service unavailable")
    return "Success after retries!"

# Test the patterns
result = await slow_operation()
print(f"Timed result: {result}")

print("\nTesting retry pattern:")
result = await flaky_operation()
print(f"Retry result: {result}")

In [ ]:
import asyncio
from typing import TypeVar, Generic, Optional
from dataclasses import dataclass
from datetime import datetime, timedelta

T = TypeVar('T')

# Pattern 6: Async cache
@dataclass
class CacheEntry(Generic[T]):
    value: T
    expires_at: datetime

class AsyncCache(Generic[T]):
    """Simple async-aware cache."""
    
    def __init__(self, ttl_seconds: float = 60.0):
        self._cache: dict[str, CacheEntry[T]] = {}
        self._ttl = timedelta(seconds=ttl_seconds)
        self._lock = asyncio.Lock()
    
    async def get(self, key: str) -> Optional[T]:
        """Get value from cache."""
        async with self._lock:
            entry = self._cache.get(key)
            if entry and datetime.now() < entry.expires_at:
                return entry.value
            return None
    
    async def set(self, key: str, value: T) -> None:
        """Set value in cache."""
        async with self._lock:
            self._cache[key] = CacheEntry(
                value=value,
                expires_at=datetime.now() + self._ttl
            )
    
    async def get_or_set(self, key: str, factory: Callable[[], Awaitable[T]]) -> T:
        """Get from cache or compute and store."""
        cached = await self.get(key)
        if cached is not None:
            print(f"Cache HIT for {key}")
            return cached
        
        print(f"Cache MISS for {key}")
        value = await factory()
        await self.set(key, value)
        return value

# Demonstrate the cache
cache: AsyncCache[dict] = AsyncCache(ttl_seconds=10)

async def expensive_fetch(user_id: int) -> dict:
    """Simulates expensive database fetch."""
    await asyncio.sleep(0.5)  # Expensive operation
    return {"id": user_id, "data": "expensive_data"}

# First call - cache miss
user = await cache.get_or_set("user:1", lambda: expensive_fetch(1))
print(f"Result: {user}")

# Second call - cache hit
user = await cache.get_or_set("user:1", lambda: expensive_fetch(1))
print(f"Result: {user}")

---

<a id="4-concurrent-tasks"></a>
## 4. Running Concurrent Tasks

### 4.1 asyncio.gather - Wait for All

Best for: Running multiple tasks and collecting all results together.

In [ ]:
import asyncio
import time

async def fetch_service(name: str, delay: float) -> dict:
    """Simulate fetching from a service."""
    await asyncio.sleep(delay)
    return {"service": name, "delay": delay}

async def demonstrate_gather():
    """Show asyncio.gather usage patterns."""
    
    # Basic gather - all results in order
    print("Basic gather:")
    start = time.perf_counter()
    
    results = await asyncio.gather(
        fetch_service("users", 1.0),
        fetch_service("orders", 0.5),
        fetch_service("products", 0.8),
    )
    
    elapsed = time.perf_counter() - start
    print(f"  Results: {results}")
    print(f"  Total time: {elapsed:.2f}s (concurrent, not 2.3s sequential)")
    
    # With return_exceptions=True - don't fail on first error
    print("\nGather with return_exceptions:")
    
    async def failing_service():
        await asyncio.sleep(0.2)
        raise ValueError("Service failed!")
    
    results = await asyncio.gather(
        fetch_service("users", 0.3),
        failing_service(),
        fetch_service("products", 0.4),
        return_exceptions=True  # Don't raise, return exceptions as results
    )
    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"  Task {i}: FAILED - {result}")
        else:
            print(f"  Task {i}: {result}")

await demonstrate_gather()

### 4.2 asyncio.wait - More Control

Best for: When you need to handle tasks as they complete or need timeout control.

In [ ]:
import asyncio

async def task_with_name(name: str, delay: float) -> str:
    """Named task for demonstration."""
    await asyncio.sleep(delay)
    return f"{name} completed"

async def demonstrate_wait():
    """Show asyncio.wait usage patterns."""
    
    tasks = [
        asyncio.create_task(task_with_name("Fast", 0.5), name="fast"),
        asyncio.create_task(task_with_name("Medium", 1.0), name="medium"),
        asyncio.create_task(task_with_name("Slow", 2.0), name="slow"),
    ]
    
    # Wait for first task to complete
    print("FIRST_COMPLETED mode:")
    done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
    print(f"  Done: {[t.get_name() for t in done]}")
    print(f"  Pending: {[t.get_name() for t in pending]}")
    
    # Wait for remaining with timeout
    print("\nWith timeout (0.6s):")
    done2, pending2 = await asyncio.wait(pending, timeout=0.6)
    print(f"  Done: {[t.get_name() for t in done2]}")
    print(f"  Still pending: {[t.get_name() for t in pending2]}")
    
    # Cancel remaining tasks
    for task in pending2:
        task.cancel()
        try:
            await task
        except asyncio.CancelledError:
            print(f"  Cancelled: {task.get_name()}")

await demonstrate_wait()

### 4.3 asyncio.as_completed - Process Results Immediately

Best for: Processing results as soon as they're available.

In [ ]:
import asyncio
import time

async def api_call(endpoint: str, delay: float) -> dict:
    """Simulate API call with variable response time."""
    await asyncio.sleep(delay)
    return {"endpoint": endpoint, "data": f"response from {endpoint}"}

async def demonstrate_as_completed():
    """Show asyncio.as_completed for streaming results."""
    
    tasks = [
        api_call("/users", 1.5),
        api_call("/orders", 0.5),
        api_call("/products", 1.0),
        api_call("/inventory", 0.3),
    ]
    
    print("Processing results as they complete:")
    start = time.perf_counter()
    
    # Results come in completion order, not submission order
    for i, coro in enumerate(asyncio.as_completed(tasks)):
        result = await coro
        elapsed = time.perf_counter() - start
        print(f"  [{elapsed:.2f}s] Result {i+1}: {result['endpoint']}")
    
    total = time.perf_counter() - start
    print(f"\nAll completed in {total:.2f}s")

await demonstrate_as_completed()

### 4.4 TaskGroup (Python 3.11+) - Structured Concurrency

Best for: Modern approach with automatic cleanup and better error handling.

In [ ]:
import asyncio
import sys

async def service_call(name: str, delay: float, should_fail: bool = False) -> str:
    """Simulated service call."""
    await asyncio.sleep(delay)
    if should_fail:
        raise RuntimeError(f"{name} failed!")
    return f"{name}: success"

async def demonstrate_taskgroup():
    """Show TaskGroup for structured concurrency."""
    
    print(f"Python version: {sys.version_info.major}.{sys.version_info.minor}")
    
    if sys.version_info >= (3, 11):
        # Successful task group
        print("\nSuccessful TaskGroup:")
        results = []
        
        async with asyncio.TaskGroup() as tg:
            # Create tasks within the group
            task1 = tg.create_task(service_call("Service A", 0.5))
            task2 = tg.create_task(service_call("Service B", 0.3))
            task3 = tg.create_task(service_call("Service C", 0.7))
        
        # All tasks are complete when we exit the context manager
        print(f"  Result 1: {task1.result()}")
        print(f"  Result 2: {task2.result()}")
        print(f"  Result 3: {task3.result()}")
        
        # TaskGroup with error - all tasks are cancelled
        print("\nTaskGroup with error (demonstrates automatic cancellation):")
        try:
            async with asyncio.TaskGroup() as tg:
                tg.create_task(service_call("Good Service", 0.5))
                tg.create_task(service_call("Bad Service", 0.2, should_fail=True))
                tg.create_task(service_call("Another Good", 0.4))
        except* RuntimeError as eg:
            print(f"  Caught errors: {[str(e) for e in eg.exceptions]}")
    else:
        print("TaskGroup requires Python 3.11+")
        print("Using gather with return_exceptions as alternative...")

await demonstrate_taskgroup()

### Comparison: When to Use Each

| Function | Use Case | Error Handling | Order |
|----------|----------|----------------|-------|
| `gather` | Collect all results | Can return exceptions | Preserves input order |
| `wait` | Need timeout/first-complete | Manual handling | Unordered sets |
| `as_completed` | Process results immediately | Per-task handling | Completion order |
| `TaskGroup` | Structured concurrency | Automatic cleanup | N/A (no return) |

---

<a id="5-async-context-managers"></a>
## 5. Async Context Managers and Iterators

### 5.1 Async Context Managers

In [ ]:
import asyncio
from contextlib import asynccontextmanager
from typing import AsyncIterator

# Method 1: Class-based async context manager
class AsyncDatabaseConnection:
    """Async database connection with context manager."""
    
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
        self.connected = False
    
    async def __aenter__(self) -> 'AsyncDatabaseConnection':
        """Async enter - establish connection."""
        print(f"Connecting to {self.connection_string}...")
        await asyncio.sleep(0.2)  # Simulate connection
        self.connected = True
        print("Connected!")
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb) -> bool:
        """Async exit - close connection."""
        print("Closing connection...")
        await asyncio.sleep(0.1)  # Simulate cleanup
        self.connected = False
        print("Connection closed!")
        return False  # Don't suppress exceptions
    
    async def query(self, sql: str) -> list:
        """Execute a query."""
        if not self.connected:
            raise RuntimeError("Not connected!")
        await asyncio.sleep(0.1)  # Simulate query
        return [{"result": f"data for: {sql}"}]

# Test class-based context manager
print("Class-based async context manager:")
async with AsyncDatabaseConnection("postgresql://localhost/db") as db:
    result = await db.query("SELECT * FROM users")
    print(f"Query result: {result}")

In [ ]:
from contextlib import asynccontextmanager

# Method 2: Decorator-based async context manager
@asynccontextmanager
async def async_timer(name: str) -> AsyncIterator[None]:
    """Context manager to time async operations."""
    import time
    start = time.perf_counter()
    print(f"[{name}] Starting...")
    try:
        yield  # The body of the 'async with' runs here
    finally:
        elapsed = time.perf_counter() - start
        print(f"[{name}] Completed in {elapsed:.4f}s")

@asynccontextmanager
async def managed_resource(resource_id: str) -> AsyncIterator[dict]:
    """Manage a resource lifecycle."""
    print(f"Acquiring resource {resource_id}")
    await asyncio.sleep(0.1)
    resource = {"id": resource_id, "status": "active"}
    
    try:
        yield resource
    except Exception as e:
        print(f"Error occurred: {e}")
        resource["status"] = "error"
        raise
    finally:
        print(f"Releasing resource {resource_id}")
        await asyncio.sleep(0.05)
        resource["status"] = "released"

# Test decorator-based context managers
print("Decorator-based async context managers:")
async with async_timer("database_operation"):
    async with managed_resource("res-123") as resource:
        print(f"Using resource: {resource}")
        await asyncio.sleep(0.3)  # Simulate work

### 5.2 Async Iterators and Generators

In [ ]:
import asyncio
from typing import AsyncIterator

# Method 1: Async generator function
async def async_range(start: int, stop: int, delay: float = 0.1) -> AsyncIterator[int]:
    """Async version of range."""
    for i in range(start, stop):
        await asyncio.sleep(delay)  # Simulate async work
        yield i

# Method 2: Class-based async iterator
class AsyncPaginator:
    """Paginate through a large dataset asynchronously."""
    
    def __init__(self, total_items: int, page_size: int = 10):
        self.total_items = total_items
        self.page_size = page_size
        self.current_page = 0
    
    def __aiter__(self) -> 'AsyncPaginator':
        return self
    
    async def __anext__(self) -> list:
        start = self.current_page * self.page_size
        if start >= self.total_items:
            raise StopAsyncIteration
        
        # Simulate fetching page from database
        await asyncio.sleep(0.1)
        
        end = min(start + self.page_size, self.total_items)
        items = list(range(start, end))
        self.current_page += 1
        
        return items

# Test async iterator
print("Async range:")
async for num in async_range(0, 5, delay=0.1):
    print(f"  Got: {num}")

print("\nAsync paginator:")
async for page in AsyncPaginator(total_items=25, page_size=10):
    print(f"  Page: {page}")

In [ ]:
# Real-world example: Streaming API responses
async def stream_events(event_source: str, count: int = 5) -> AsyncIterator[dict]:
    """Stream events from a source."""
    for i in range(count):
        await asyncio.sleep(0.2)  # Simulate waiting for event
        yield {
            "id": i,
            "source": event_source,
            "data": f"Event {i} data",
            "timestamp": asyncio.get_event_loop().time()
        }

# Async comprehension
print("Streaming events with async comprehension:")
events = [event async for event in stream_events("sensor", count=3)]
for event in events:
    print(f"  {event}")

# Async filtering
print("\nFiltered events (even IDs only):")
even_events = [e async for e in stream_events("filter-test", count=5) if e['id'] % 2 == 0]
for event in even_events:
    print(f"  {event}")

---

<a id="6-error-handling"></a>
## 6. Error Handling in Async Code

### 6.1 Basic Exception Handling

In [ ]:
import asyncio
from typing import Optional

class ServiceError(Exception):
    """Custom service error."""
    pass

async def unreliable_service(success_rate: float = 0.5) -> dict:
    """Simulates an unreliable service."""
    import random
    await asyncio.sleep(0.1)
    
    if random.random() > success_rate:
        raise ServiceError("Service temporarily unavailable")
    
    return {"status": "success", "data": "valuable data"}

async def call_with_handling() -> Optional[dict]:
    """Call service with proper error handling."""
    try:
        result = await unreliable_service(success_rate=0.3)
        return result
    except ServiceError as e:
        print(f"Service error: {e}")
        return None
    except asyncio.CancelledError:
        print("Task was cancelled")
        raise  # Always re-raise CancelledError
    except Exception as e:
        print(f"Unexpected error: {type(e).__name__}: {e}")
        return None

# Test error handling
print("Testing error handling (multiple attempts):")
for i in range(5):
    result = await call_with_handling()
    print(f"  Attempt {i+1}: {result}")

### 6.2 Handling Errors in Concurrent Tasks

In [ ]:
import asyncio
from dataclasses import dataclass
from typing import Union, List

@dataclass
class Success:
    value: any

@dataclass  
class Failure:
    error: Exception

Result = Union[Success, Failure]

async def task_a() -> str:
    await asyncio.sleep(0.2)
    return "A succeeded"

async def task_b() -> str:
    await asyncio.sleep(0.1)
    raise ValueError("B failed!")

async def task_c() -> str:
    await asyncio.sleep(0.3)
    return "C succeeded"

async def safe_gather(*coros) -> List[Result]:
    """Gather that returns Success/Failure for each task."""
    results = await asyncio.gather(*coros, return_exceptions=True)
    return [
        Failure(r) if isinstance(r, Exception) else Success(r)
        for r in results
    ]

# Demonstrate safe gathering
print("Safe gather with mixed success/failure:")
results = await safe_gather(task_a(), task_b(), task_c())

for i, result in enumerate(results):
    match result:
        case Success(value):
            print(f"  Task {i}: ✓ {value}")
        case Failure(error):
            print(f"  Task {i}: ✗ {type(error).__name__}: {error}")

### 6.3 Timeout Handling

In [ ]:
import asyncio
from typing import TypeVar, Optional

T = TypeVar('T')

async def slow_operation(duration: float) -> str:
    """An operation that takes a while."""
    await asyncio.sleep(duration)
    return f"Completed after {duration}s"

async def with_timeout(
    coro,
    timeout: float,
    default: Optional[T] = None
) -> Optional[T]:
    """Execute coroutine with timeout, return default on timeout."""
    try:
        return await asyncio.wait_for(coro, timeout=timeout)
    except asyncio.TimeoutError:
        print(f"Operation timed out after {timeout}s")
        return default

# Test timeout handling
print("Timeout handling:")

# This should succeed
result = await with_timeout(slow_operation(0.5), timeout=1.0)
print(f"  Fast operation: {result}")

# This should timeout
result = await with_timeout(slow_operation(2.0), timeout=0.5, default="TIMEOUT")
print(f"  Slow operation: {result}")

# Using asyncio.timeout context manager (Python 3.11+)
import sys
if sys.version_info >= (3, 11):
    print("\nUsing asyncio.timeout context manager:")
    try:
        async with asyncio.timeout(0.3):
            result = await slow_operation(1.0)
    except TimeoutError:
        print("  Context manager timeout triggered")

---

<a id="7-common-pitfalls"></a>
## 7. Common Pitfalls

### 7.1 Blocking the Event Loop

**The #1 async mistake:** Calling blocking code in async functions.

In [ ]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

# ❌ BAD: Blocking the event loop
async def bad_async_function():
    """This blocks the entire event loop!"""
    time.sleep(1)  # ❌ Blocking call!
    return "Done"

# ✅ GOOD: Use asyncio.sleep for delays
async def good_async_function():
    """Non-blocking async function."""
    await asyncio.sleep(1)  # ✅ Non-blocking
    return "Done"

# ✅ GOOD: Run blocking code in thread pool
def cpu_intensive_work(n: int) -> int:
    """Simulate CPU-bound work."""
    total = 0
    for i in range(n):
        total += i * i
    return total

async def properly_wrapped_blocking():
    """Run blocking code without blocking the event loop."""
    loop = asyncio.get_running_loop()
    
    # Run in default thread pool executor
    result = await loop.run_in_executor(None, cpu_intensive_work, 100000)
    return result

# Demonstrate the difference
print("Running blocking code properly:")

async def demonstrate_blocking():
    start = time.perf_counter()
    
    # Run multiple CPU tasks concurrently using thread pool
    results = await asyncio.gather(
        properly_wrapped_blocking(),
        properly_wrapped_blocking(),
        properly_wrapped_blocking(),
    )
    
    elapsed = time.perf_counter() - start
    print(f"  Completed 3 CPU tasks in {elapsed:.2f}s (concurrent)")
    return results

await demonstrate_blocking()

### 7.2 Forgetting to Await

In [ ]:
import asyncio
import warnings

async def fetch_data() -> dict:
    await asyncio.sleep(0.1)
    return {"data": "result"}

async def demonstrate_missing_await():
    """Show what happens when you forget to await."""
    
    # ❌ BAD: Forgot to await - returns coroutine object, not result!
    result_bad = fetch_data()  # No await!
    print(f"Without await: {result_bad}")
    print(f"Type: {type(result_bad)}")
    
    # Clean up the unawaited coroutine to avoid warning
    await result_bad
    
    # ✅ GOOD: Properly awaited
    result_good = await fetch_data()
    print(f"\nWith await: {result_good}")
    print(f"Type: {type(result_good)}")

await demonstrate_missing_await()

### 7.3 Task Cancellation

In [ ]:
import asyncio

async def cancellable_task(name: str):
    """A task that handles cancellation gracefully."""
    try:
        print(f"[{name}] Starting work...")
        for i in range(10):
            await asyncio.sleep(0.2)
            print(f"[{name}] Step {i+1}/10")
        print(f"[{name}] Completed!")
        return f"{name} finished"
    except asyncio.CancelledError:
        print(f"[{name}] Cancelled! Cleaning up...")
        # Perform cleanup here
        await asyncio.sleep(0.1)  # Simulate cleanup
        print(f"[{name}] Cleanup complete")
        raise  # Always re-raise CancelledError!

async def demonstrate_cancellation():
    """Show proper task cancellation."""
    
    # Start a task
    task = asyncio.create_task(cancellable_task("Worker"))
    
    # Let it run for a bit
    await asyncio.sleep(0.5)
    
    # Cancel the task
    print("\n--- Requesting cancellation ---\n")
    task.cancel()
    
    # Wait for cancellation to complete
    try:
        await task
    except asyncio.CancelledError:
        print("\nTask cancellation confirmed")
    
    print(f"Task state: cancelled={task.cancelled()}")

await demonstrate_cancellation()

### 7.4 Fire-and-Forget Tasks (Memory Leaks)

In [ ]:
import asyncio
from typing import Set

# ❌ BAD: Fire and forget without tracking
async def bad_fire_and_forget():
    """Tasks can be garbage collected before completion!"""
    asyncio.create_task(some_background_work())  # ❌ Not tracked!

# ✅ GOOD: Track background tasks
background_tasks: Set[asyncio.Task] = set()

async def some_background_work():
    await asyncio.sleep(0.5)
    print("  Background work completed!")

def create_background_task(coro):
    """Create a tracked background task."""
    task = asyncio.create_task(coro)
    background_tasks.add(task)
    task.add_done_callback(background_tasks.discard)
    return task

async def demonstrate_tracked_tasks():
    """Show proper background task management."""
    print("Creating background tasks:")
    
    # Create tracked tasks
    create_background_task(some_background_work())
    create_background_task(some_background_work())
    
    print(f"  Active background tasks: {len(background_tasks)}")
    
    # Wait for all background tasks
    if background_tasks:
        await asyncio.gather(*background_tasks)
    
    print(f"  Active background tasks: {len(background_tasks)}")

await demonstrate_tracked_tasks()

---

<a id="8-async-ecosystem"></a>
## 8. Async Libraries Ecosystem

### Overview of Major Async Libraries

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        ASYNC PYTHON ECOSYSTEM                               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  HTTP Clients           Databases              Caching/Queues              │
│  ───────────           ─────────              ──────────────               │
│  • httpx               • asyncpg (PostgreSQL) • aioredis                   │
│  • aiohttp             • aiomysql             • arq (job queues)           │
│                        • motor (MongoDB)      • aio-pika (RabbitMQ)        │
│                        • aiosqlite                                         │
│                                                                             │
│  Web Frameworks         ORMs                  Other                        │
│  ──────────────        ────                  ─────                         │
│  • FastAPI             • SQLAlchemy 2.0      • aiofiles                    │
│  • Starlette           • Tortoise-ORM        • aioboto3 (AWS)              │
│  • Sanic               • SQLModel            • asyncssh                    │
│  • Quart                                                                   │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 8.1 HTTP Clients: httpx

In [ ]:
# Example: Using httpx for async HTTP requests
# Note: Install with `pip install httpx`

import asyncio

# Simulated httpx-like async client example
# (In real code, you would import httpx)

class MockAsyncClient:
    """Mock async HTTP client for demonstration."""
    
    async def __aenter__(self):
        print("Opening HTTP client connection pool")
        return self
    
    async def __aexit__(self, *args):
        print("Closing HTTP client connection pool")
    
    async def get(self, url: str) -> dict:
        await asyncio.sleep(0.2)  # Simulate network latency
        return {"status": 200, "url": url, "data": f"Response from {url}"}

async def httpx_example():
    """Demonstrate async HTTP client patterns."""
    
    # Using async context manager for connection pooling
    async with MockAsyncClient() as client:
        # Single request
        response = await client.get("https://api.example.com/users")
        print(f"Single request: {response}")
        
        # Concurrent requests
        urls = [
            "https://api.example.com/users",
            "https://api.example.com/orders",
            "https://api.example.com/products",
        ]
        
        responses = await asyncio.gather(
            *[client.get(url) for url in urls]
        )
        print(f"\nConcurrent requests:")
        for resp in responses:
            print(f"  {resp['url']}: {resp['status']}")

await httpx_example()

# Real httpx code would look like:
print("\n--- Real httpx code example ---")
print("""
import httpx

async with httpx.AsyncClient() as client:
    # GET request
    response = await client.get('https://api.github.com/users/octocat')
    
    # POST request with JSON
    response = await client.post(
        'https://api.example.com/data',
        json={'key': 'value'},
        headers={'Authorization': 'Bearer token'}
    )
    
    # Concurrent requests
    responses = await asyncio.gather(
        client.get('https://api.example.com/a'),
        client.get('https://api.example.com/b'),
    )
""")

### 8.2 Database: asyncpg (PostgreSQL)

In [ ]:
# Example: Using asyncpg for async PostgreSQL queries
# Note: Install with `pip install asyncpg`

print("--- asyncpg pattern examples ---")
print("""
import asyncpg

# Connection pool pattern (recommended for web apps)
async def create_pool():
    return await asyncpg.create_pool(
        host='localhost',
        database='mydb',
        user='user',
        password='password',
        min_size=5,
        max_size=20
    )

async def query_users(pool: asyncpg.Pool):
    async with pool.acquire() as conn:
        # Single row
        user = await conn.fetchrow(
            'SELECT * FROM users WHERE id = $1',
            user_id
        )
        
        # Multiple rows
        users = await conn.fetch(
            'SELECT * FROM users WHERE active = $1',
            True
        )
        
        # Transaction
        async with conn.transaction():
            await conn.execute(
                'INSERT INTO users(name, email) VALUES($1, $2)',
                'John', 'john@example.com'
            )
            await conn.execute(
                'UPDATE accounts SET balance = balance - $1 WHERE user_id = $2',
                100, user_id
            )

# Copy for bulk inserts (very fast!)
async def bulk_insert(pool: asyncpg.Pool, records: list):
    async with pool.acquire() as conn:
        await conn.copy_records_to_table(
            'users',
            records=records,
            columns=['name', 'email', 'created_at']
        )
""")

### 8.3 Caching: Redis with aioredis

In [ ]:
# Example: Using redis-py (which includes async support)
# Note: Install with `pip install redis`

print("--- redis async pattern examples ---")
print("""
import redis.asyncio as redis

# Connection pool
pool = redis.ConnectionPool.from_url(
    'redis://localhost:6379',
    max_connections=10
)

async def get_redis():
    return redis.Redis(connection_pool=pool)

async def cache_example():
    r = await get_redis()
    
    # Basic operations
    await r.set('key', 'value', ex=3600)  # 1 hour expiry
    value = await r.get('key')
    
    # Increment counter
    await r.incr('page_views')
    
    # Hash operations
    await r.hset('user:1', mapping={
        'name': 'John',
        'email': 'john@example.com'
    })
    user = await r.hgetall('user:1')
    
    # List operations (for queues)
    await r.lpush('task_queue', 'task_data')
    task = await r.brpop('task_queue', timeout=5)  # Blocking pop
    
    # Pub/Sub
    pubsub = r.pubsub()
    await pubsub.subscribe('channel')
    
    async for message in pubsub.listen():
        if message['type'] == 'message':
            print(f"Received: {message['data']}")

# Cache decorator pattern
def redis_cache(expire: int = 300):
    def decorator(func):
        async def wrapper(*args, **kwargs):
            r = await get_redis()
            key = f"{func.__name__}:{args}:{kwargs}"
            
            cached = await r.get(key)
            if cached:
                return json.loads(cached)
            
            result = await func(*args, **kwargs)
            await r.set(key, json.dumps(result), ex=expire)
            return result
        return wrapper
    return decorator
""")

---

<a id="9-fastapi-patterns"></a>
## 9. FastAPI Async Patterns

### 9.1 Basic Async Endpoints

In [ ]:
# FastAPI async patterns demonstration
# Note: Install with `pip install fastapi uvicorn`

fastapi_code = '''
from fastapi import FastAPI, Depends, HTTPException
from typing import List, Optional
import asyncio

app = FastAPI()

# Basic async endpoint
@app.get("/users/{user_id}")
async def get_user(user_id: int):
    """Async endpoint - use for I/O bound operations."""
    user = await fetch_user_from_db(user_id)
    return user

# Sync endpoint (runs in thread pool)
@app.get("/compute/{n}")
def compute_heavy(n: int):
    """Sync endpoint - FastAPI runs this in thread pool automatically."""
    # CPU-bound work
    result = sum(i * i for i in range(n))
    return {"result": result}

# Aggregating multiple services
@app.get("/dashboard/{user_id}")
async def get_dashboard(user_id: int):
    """Fetch from multiple services concurrently."""
    user, orders, notifications = await asyncio.gather(
        fetch_user(user_id),
        fetch_orders(user_id),
        fetch_notifications(user_id),
    )
    return {
        "user": user,
        "orders": orders,
        "notifications": notifications
    }
'''

print("FastAPI Basic Patterns:")
print(fastapi_code)

### 9.2 Dependency Injection with Async

In [ ]:
fastapi_deps = '''
from fastapi import FastAPI, Depends
from contextlib import asynccontextmanager
from typing import AsyncGenerator

# Async database session dependency
async def get_db_session() -> AsyncGenerator[DatabaseSession, None]:
    """Yield a database session and ensure cleanup."""
    session = await create_session()
    try:
        yield session
    finally:
        await session.close()

# Async HTTP client dependency
async def get_http_client() -> AsyncGenerator[httpx.AsyncClient, None]:
    """Yield an HTTP client."""
    async with httpx.AsyncClient() as client:
        yield client

# Using dependencies
@app.get("/users/{user_id}")
async def get_user(
    user_id: int,
    db: DatabaseSession = Depends(get_db_session),
    http: httpx.AsyncClient = Depends(get_http_client)
):
    user = await db.query(User).get(user_id)
    
    # Fetch additional data from external API
    external_data = await http.get(f"https://api.example.com/users/{user_id}")
    
    return {"user": user, "external": external_data.json()}

# Lifespan context for startup/shutdown
@asynccontextmanager
async def lifespan(app: FastAPI):
    """Manage application lifecycle."""
    # Startup
    app.state.db_pool = await create_pool()
    app.state.redis = await create_redis_client()
    
    yield  # Application runs here
    
    # Shutdown
    await app.state.db_pool.close()
    await app.state.redis.close()

app = FastAPI(lifespan=lifespan)
'''

print("FastAPI Dependency Injection:")
print(fastapi_deps)

### 9.3 Background Tasks

In [ ]:
fastapi_background = '''
from fastapi import FastAPI, BackgroundTasks

app = FastAPI()

async def send_email(email: str, message: str):
    """Send email in background."""
    await asyncio.sleep(5)  # Simulate email sending
    print(f"Email sent to {email}")

async def log_action(user_id: int, action: str):
    """Log user action to analytics."""
    await asyncio.sleep(1)  # Simulate logging
    print(f"Logged {action} for user {user_id}")

@app.post("/users/")
async def create_user(
    user: UserCreate,
    background_tasks: BackgroundTasks
):
    """Create user and send welcome email in background."""
    # Create user immediately
    new_user = await db.create_user(user)
    
    # Schedule background tasks (don't block response)
    background_tasks.add_task(
        send_email, 
        new_user.email, 
        "Welcome to our platform!"
    )
    background_tasks.add_task(
        log_action,
        new_user.id,
        "user_created"
    )
    
    # Return immediately
    return {"id": new_user.id, "status": "created"}

# For more complex background jobs, use:
# - Celery with async support
# - arq (async job queue)
# - dramatiq with async
'''

print("FastAPI Background Tasks:")
print(fastapi_background)

---

<a id="10-testing-async"></a>
## 10. Testing Async Code

### 10.1 Testing with pytest-asyncio

In [ ]:
pytest_examples = '''
# Install: pip install pytest pytest-asyncio

import pytest
import asyncio

# Mark all tests in module as async
pytestmark = pytest.mark.asyncio

# Basic async test
async def test_async_function():
    result = await my_async_function()
    assert result == expected_value

# Test with fixtures
@pytest.fixture
async def db_session():
    """Async fixture for database session."""
    session = await create_test_session()
    yield session
    await session.rollback()
    await session.close()

async def test_create_user(db_session):
    user = await create_user(db_session, name="Test User")
    assert user.id is not None
    assert user.name == "Test User"

# Test exceptions
async def test_raises_exception():
    with pytest.raises(ValueError):
        await function_that_raises()

# Test timeouts
async def test_with_timeout():
    async with asyncio.timeout(1.0):
        result = await potentially_slow_function()
    assert result is not None

# Test concurrent execution
async def test_concurrent_operations():
    results = await asyncio.gather(
        operation_a(),
        operation_b(),
        operation_c(),
    )
    assert all(r.success for r in results)

# Mocking async functions
from unittest.mock import AsyncMock, patch

async def test_with_mock():
    mock_fetch = AsyncMock(return_value={"id": 1, "name": "Test"})
    
    with patch("mymodule.fetch_user", mock_fetch):
        result = await get_user_data(1)
    
    mock_fetch.assert_called_once_with(1)
    assert result["name"] == "Test"
'''

print("pytest-asyncio Testing Patterns:")
print(pytest_examples)

### 10.2 Testing FastAPI Async Endpoints

In [ ]:
fastapi_testing = '''
# Install: pip install httpx pytest-asyncio

import pytest
from httpx import AsyncClient, ASGITransport
from myapp import app

# Async test client fixture
@pytest.fixture
async def client():
    async with AsyncClient(
        transport=ASGITransport(app=app),
        base_url="http://test"
    ) as client:
        yield client

# Test GET endpoint
@pytest.mark.asyncio
async def test_get_users(client):
    response = await client.get("/users/")
    assert response.status_code == 200
    assert isinstance(response.json(), list)

# Test POST endpoint
@pytest.mark.asyncio
async def test_create_user(client):
    response = await client.post(
        "/users/",
        json={"name": "Test User", "email": "test@example.com"}
    )
    assert response.status_code == 201
    data = response.json()
    assert data["name"] == "Test User"

# Test with authentication
@pytest.mark.asyncio
async def test_protected_endpoint(client):
    # First, login to get token
    login_response = await client.post(
        "/auth/login",
        json={"username": "test", "password": "password"}
    )
    token = login_response.json()["access_token"]
    
    # Access protected endpoint
    response = await client.get(
        "/users/me",
        headers={"Authorization": f"Bearer {token}"}
    )
    assert response.status_code == 200

# Test with database override
@pytest.fixture
async def override_db():
    async def get_test_db():
        # Return test database session
        async with TestSession() as session:
            yield session
    
    app.dependency_overrides[get_db] = get_test_db
    yield
    app.dependency_overrides.clear()

@pytest.mark.asyncio
async def test_with_test_db(client, override_db):
    response = await client.get("/users/1")
    assert response.status_code == 200
'''

print("FastAPI Async Testing Patterns:")
print(fastapi_testing)

---

<a id="11-when-not-to-use"></a>
## 11. When NOT to Use Async

### Decision Framework

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    WHEN TO USE ASYNC - DECISION TREE                        │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Is your workload primarily I/O bound?                                      │
│  │                                                                          │
│  ├─ YES: Do you need to handle many concurrent connections?                 │
│  │   │                                                                      │
│  │   ├─ YES: ✅ USE ASYNC                                                   │
│  │   │   • Web servers (FastAPI, Starlette)                                │
│  │   │   • API gateways                                                     │
│  │   │   • Websocket servers                                                │
│  │   │   • Microservices                                                    │
│  │   │                                                                      │
│  │   └─ NO: Consider complexity trade-off                                   │
│  │       • Simple scripts → sync may be simpler                             │
│  │       • Few concurrent tasks → sync is fine                              │
│  │                                                                          │
│  └─ NO (CPU bound):                                                         │
│      │                                                                      │
│      └─ ❌ DON'T USE ASYNC ALONE                                            │
│          • Use multiprocessing for CPU parallelism                          │
│          • Use ProcessPoolExecutor                                          │
│          • Consider async + multiprocessing hybrid                          │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# When NOT to use async - Examples

import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

# ❌ BAD: Using async for CPU-bound work
async def bad_cpu_bound():
    """This does NOT benefit from async!"""
    # CPU work blocks the event loop
    result = sum(i * i for i in range(1000000))
    return result

# ✅ GOOD: Use process pool for CPU-bound work
# NOTE: In production scripts, use ProcessPoolExecutor with if __name__ == "__main__" guard
# Here we use ThreadPoolExecutor for demonstration (works in notebooks)
def cpu_work(n: int) -> int:
    """CPU-bound function."""
    return sum(i * i for i in range(n))

async def good_cpu_bound():
    """Offload CPU work to thread/process pool."""
    loop = asyncio.get_running_loop()
    
    # Using ThreadPoolExecutor for notebook compatibility
    # In production: use ProcessPoolExecutor for true parallelism
    with ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, cpu_work, 1000000)
    
    return result

print("CPU-bound work patterns demonstrated")
print("\nFor CPU-bound work, prefer:")
print("  1. multiprocessing.Pool (with if __name__ == '__main__' guard)")
print("  2. concurrent.futures.ProcessPoolExecutor (same guard needed)")
print("  3. Libraries like Dask, Ray for distributed computing")
print("\nNote: ProcessPoolExecutor requires picklable functions,")
print("      which doesn't work directly in notebooks on Windows.")


### Cases Where Sync is Preferred

| Scenario | Why Sync is Better |
|----------|--------------------|
| Simple scripts | Async adds complexity without benefit |
| CPU-bound processing | Async doesn't parallelize CPU work |
| Single database query | No concurrency to exploit |
| Legacy codebase | Mixing sync/async adds complexity |
| Libraries lack async support | Wrapping sync code defeats purpose |

In [ ]:
# Hybrid approach: Async for I/O, ThreadPool for CPU simulation
# NOTE: In production, use ProcessPoolExecutor with proper __main__ guard for true CPU parallelism

import asyncio
from concurrent.futures import ThreadPoolExecutor
from typing import List

def process_image(image_data: bytes) -> bytes:
    """CPU-intensive image processing (simulated)."""
    import time
    time.sleep(0.1)  # Simulate work
    return b"processed_" + image_data[:10]

async def fetch_image(url: str) -> bytes:
    """I/O-bound: Fetch image from URL."""
    await asyncio.sleep(0.05)  # Simulate network I/O
    return f"image_data_from_{url}".encode()

async def process_images_hybrid(urls: List[str]) -> List[bytes]:
    """Hybrid approach: async for I/O, thread pool for CPU simulation."""
    loop = asyncio.get_running_loop()
    
    # Step 1: Fetch all images concurrently (I/O bound)
    print("Fetching images concurrently...")
    images = await asyncio.gather(*[fetch_image(url) for url in urls])
    
    # Step 2: Process images in parallel
    # Using ThreadPoolExecutor for notebook compatibility
    # In production with real CPU work: use ProcessPoolExecutor
    print("Processing images in parallel...")
    with ThreadPoolExecutor(max_workers=4) as pool:
        processed = await asyncio.gather(
            *[loop.run_in_executor(pool, process_image, img) for img in images]
        )
    
    return list(processed)

# Demonstrate hybrid approach
urls = [f"https://example.com/image_{i}.jpg" for i in range(5)]
results = await process_images_hybrid(urls)
print(f"\nProcessed {len(results)} images")
for i, result in enumerate(results):
    print(f"  Image {i}: {result.decode()[:30]}...")

---

## Summary: Async Programming Best Practices

### Do's ✅

1. **Use async for I/O-bound operations**: Database queries, HTTP requests, file I/O
2. **Use `asyncio.gather` for concurrent operations**: When you need all results
3. **Use `asyncio.TaskGroup` (3.11+)**: For structured concurrency with proper cleanup
4. **Handle `CancelledError` properly**: Always re-raise after cleanup
5. **Use connection pools**: For databases and HTTP clients
6. **Test with pytest-asyncio**: Use proper async testing patterns

### Don'ts ❌

1. **Don't block the event loop**: No `time.sleep()`, no synchronous I/O
2. **Don't forget to await**: Unawaited coroutines don't execute
3. **Don't use async for CPU-bound work**: Use multiprocessing instead
4. **Don't create tasks without tracking**: Leads to memory leaks
5. **Don't mix sync and async carelessly**: Leads to blocking issues

### Performance Tips 🚀

1. **Batch database operations**: Use bulk inserts/updates
2. **Use connection pooling**: Avoid connection overhead
3. **Limit concurrency**: Use `asyncio.Semaphore` to prevent overwhelming resources
4. **Profile your code**: Use `py-spy` or `scalene` for async profiling
5. **Consider uvloop**: Drop-in replacement for faster event loop

In [ ]:
# Final example: Production-ready async service pattern

import asyncio
from typing import List, Optional
from dataclasses import dataclass
from contextlib import asynccontextmanager

@dataclass
class User:
    id: int
    name: str
    email: str

class UserService:
    """Production-ready async user service."""
    
    def __init__(self, max_concurrent: int = 10):
        self._semaphore = asyncio.Semaphore(max_concurrent)
        self._cache: dict[int, User] = {}
    
    async def get_user(self, user_id: int) -> Optional[User]:
        """Get user with caching and concurrency control."""
        # Check cache first
        if user_id in self._cache:
            return self._cache[user_id]
        
        # Limit concurrent database calls
        async with self._semaphore:
            # Double-check cache (another task might have populated it)
            if user_id in self._cache:
                return self._cache[user_id]
            
            # Fetch from database
            user = await self._fetch_from_db(user_id)
            if user:
                self._cache[user_id] = user
            return user
    
    async def get_users(self, user_ids: List[int]) -> List[User]:
        """Get multiple users concurrently."""
        tasks = [self.get_user(uid) for uid in user_ids]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        users = []
        for result in results:
            if isinstance(result, User):
                users.append(result)
            elif isinstance(result, Exception):
                # Log error but continue
                print(f"Error fetching user: {result}")
        
        return users
    
    async def _fetch_from_db(self, user_id: int) -> Optional[User]:
        """Simulate database fetch."""
        await asyncio.sleep(0.1)  # Simulate DB query
        return User(id=user_id, name=f"User_{user_id}", email=f"user{user_id}@example.com")

# Demonstrate the service
service = UserService(max_concurrent=5)

print("Fetching users with production patterns:")
users = await service.get_users([1, 2, 3, 4, 5])
for user in users:
    print(f"  {user}")

print("\n✅ Async programming notebook complete!")